## How to get high score using MMBT and CLIP in Hateful Memes Competition

#### Use CLIP as a feature encoder for Multimodal Bitransformer and make MMBT really work with Huggingface Transformers to get surprisingly high accuracy

The additional stage of <a href=https://www.drivendata.org/competitions/64/hateful-memes/>Hateful Memes Competition</a> from Facebook ended a few months ago. My team was lucky enough to take part in this competition and even get pretty good results (we took tenth place). How we did it and what methods we used - I'll tell you in this article. 

## Problem description

At first glance, the problem that had to be solved in the competition is quite simple - to determine whether a meme is hateful or not using text and image data from it. In reality, the problem is complicated by the many ambiguities inherent in our speech, as well as by the presence of sarcasm and irony, with the definition of which neural networks have problems. You can read more about the competition and the tasks it posed in the corresponding <a href=https://arxiv.org/pdf/2005.04790v2.pdf>paper</a>. 

<img src=https://drivendata-public-assets.s3.amazonaws.com/memes-overview.png />
Image from <a href=https://www.drivendata.org/competitions/64/hateful-memes/>DrivenData</a>

## Data Overview

During the competition, a downloadable zip file was provided. Now competition data can be found at this <a href=https://hatefulmemeschallenge.com/>link</a>.

The zip file includes a folder with images and several json files with images annotations.

<b>img/</b> folder contains all the images of the challenge dataset including train, dev and test split. The images are named \<id>.png, where \<id> is a unique 5 digit number.  

**train.jsonl, dev_seen.jsonl, dev_unseen.jsonl** — json files where each line has a dictionary of key-value pairs of data about the images. The dictionary includes

- **id** The unique identifier between the img directory and the .jsonl files, e.g., "id": 13894.
- **img** The actual meme filename, e.g., "img": img/13894.png, note that the filename contains the img directory described above, and that the filename stem is the id.
- **text** The raw text string embedded in the meme image, e.g., img/13894.png has "text": "putting bows on your pet"
- **label** where 1 -> "hateful" and 0 -> "non-hateful"


For example:

{"id":23058,"img":"img\/23058.png","label":0,"text":"don't be afraid to love again everyone is not like your ex"}

**test_seen.jsnol** includes mentioned keys, except <b>label</b>.

## Performance metric

Model performance and leaderboard rankings were determined using the **AUC ROC** or the Area Under the Curve of the Receiver Operating Characteristic. The metric measures how well your binary classifier discriminates between the classes as its decision threshold is varied. 

<img src=images/auroc.png />

Another metric was the accuracy of  predictions, given by the ratio of correct predictions to the total number of predictions made. 

<img src=images/accuracy.png />

## Our approach

There are many models and frameworks for working with multimodal data, the one that stands out the most is <a href=https://mmf.sh/>MMF</a> from Facebook. <a href=https://mmf.sh/>MMF</a> provides a simple interface for accessing many powerful multimodal models. But we, as big fans of <a href=https://github.com/huggingface/transformers>Huggingface Transformers</a>, decided not to go the easy way. We decided to find out what multimodal models are available in <a href=https://github.com/huggingface/transformers>Transformers</a> and how to get the most out of them. It turned out that only one model of this kind is currently available in Transformers - <b><a href="https://huggingface.co/transformers/summary.html#multimodal-models">Multimodal Bitransformer (MMBT)</a></b>. Making this model work was not so easy, but this made the task only more interesting. 

<img width='700px' src='images/MMBT.png'/>
MMBT architecture, from <a href='https://arxiv.org/abs/1909.02950'>Supervised Multimodal Bitransformers for Classifying Images and Text paper</a>

MMBT fuses information from text and image encoders. <a href='https://huggingface.co/transformers/model_doc/bert.html'>BERT</a> is used as text encoder and <a href='https://pytorch.org/hub/pytorch_vision_resnet/'>ResNet</a> as image encoder. We took advantage of MMBT architecture flexibility and replaced ResNet with <a href=https://github.com/openai/CLIP>CLIP</a> for image encoding.  <b>CLIP</b> pre-trains an image encoder and a text encoder to predict which images were paired with which texts in a dataset. Our assumption was that features from CLIP are more versatile and better suited for a multimodal domain.

<img width='700px' src='https://raw.githubusercontent.com/openai/CLIP/main/CLIP.png'/>
Summary of CLIP model’s approach, from <a href='https://arxiv.org/abs/2103.00020'>Learning Transferable Visual Models From Natural Language Supervision paper</a>

For text encoding we used <a href=https://huggingface.co/Hate-speech-CNERG/bert-base-uncased-hatexplain>bert-base-uncased-hatexplain</a> model which is available in Huggingface Hub. This model was created for hatespeech detection in English, so in our case features from it are better than from <a href=https://huggingface.co/bert-base-uncased>bert-base-uncased</a> that was used in MMBT initially. 

Final MMBT model was finetuned on a train dataset and validated on dev_seen dataset.

We also augmented texts in train dataset using controlled <a href="https://huggingface.co/transformers/model_doc/gpt2.html">GPT-2</a> and <a href="https://github.com/dsfsi/textaugment#eda-easy-data-augmentation-techniques-for-boosting-performance-on-text-classification-tasks">Easy Data Augmentation</a> method. This increased the accuracy of our model by an additional few percent. Augmentation is beyond the scope of this article, and I will probably write about it separately if you find this article and our approach interesting. 

## Implementation

There will be a lot of code in this section of the article, but I will try to explain all the important parts in detail for a better understanding.

First, let's import the required libraries. We will need

- Transformers version >=4.8.2
- Pytorch version 1.8.1
- torchvision 0.9.1
- scikit-learn 0.23.2 
- Pillow >=8.2.0
- tqdm >= 4.60.0
- matplotlib >= 3.3.4
- numpy >=1.19.5
- <a href=https://github.com/openai/CLIP>CLIP</a> (that can be installed from repository). 

CLIP is now <a href=https://huggingface.co/transformers/model_doc/clip.html>accesible</a> directly in Huggingface Transformers, but at the time of the implementation of our approach it was not there yet. To get the most out of our model, we also used the <a href=https://github.com/facebookresearch/madgrad>MADGRAD</a> optimizer. 

In [ ]:
!pip install madgrad

In [ ]:
# !pip install clip

In [ ]:
import json
import os
from collections import Counter
import random
import numpy as np

import torch
import torch.nn as nn
# import torchvision
# import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from madgrad import MADGRAD

from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
from transformers import CLIPProcessor, CLIPModel
# import clip
import pickle

from matplotlib import pyplot as plt
from tqdm.notebook import tqdm

import copy
import time
import transformers
from transformers import (
    AutoConfig,
    AutoModel,
    AutoTokenizer,
    MMBTConfig,
    MMBTModel,
    MMBTForClassification,
    get_linear_schedule_with_warmup,
)

In [ ]:
import torchvision.transforms as transforms

Create a variable with available device, which will do all needed computations. We will need a GPU, so our device is CUDA.

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

Load CLIP model and needed preprocessing.

In [ ]:
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git

In [ ]:
import clip

clip.available_models()


In [ ]:
clip_model, preprocess = clip.load("RN50x4", device=device, jit=False)


Freeze weights of CLIP feature encoder, as we will not finetune it. 

In [ ]:
# for p in clip_model.parameters():
#     p.requires_grad = False

Initialize needed variables.

In [ ]:
num_image_embeds = 5
num_labels = 1
gradient_accumulation_steps = 20
data_dir = '/kaggle/input/memedata/hateful_memes'
max_seq_length = 80 
max_grad_norm = 0.5
train_batch_size = 16
eval_batch_size = 16
image_encoder_size = 256
image_features_size = 512
num_train_epochs = 20

Create a function that will prepare an image for CLIP encoder in a special manner. This function will split image into three tiles (by height or width, depending on the aspect ratio of the image). Finally we will get four vectors after encoding (one vector for each tile and one vector for whole image that was padded to square).

In [ ]:
def slice_image(im, desired_size):
    '''
    Resize and slice image
    '''
    old_size = im.size
    ratio = float(desired_size)/min(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])
    im = im.resize(new_size, Image.LANCZOS)    
    ar = np.array(im)
    images = []
    if ar.shape[0] < ar.shape[1]:
        middle = ar.shape[1] // 2
        half = desired_size // 2
        
        images.append(Image.fromarray(ar[:, :desired_size]))
        images.append(Image.fromarray(ar[:, middle-half:middle+half]))
        images.append(Image.fromarray(ar[:, ar.shape[1]-desired_size:ar.shape[1]]))
    else:
        middle = ar.shape[0] // 2
        half = desired_size // 2
        
        images.append(Image.fromarray(ar[:desired_size, :]))
        images.append(Image.fromarray(ar[middle-half:middle+half, :]))
        images.append(Image.fromarray(ar[ar.shape[0]-desired_size:ar.shape[0], :]))

    return images

In [ ]:
 def resize_pad_image(im, desired_size):
    '''
    Resize and pad image to a desired size
    '''
    old_size = im.size
    ratio = float(desired_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])
    im = im.resize(new_size, Image.LANCZOS)

    # create a new image and paste the resized on it
    new_im = Image.new("RGB", (desired_size, desired_size))
    new_im.paste(im, ((desired_size-new_size[0])//2,
                        (desired_size-new_size[1])//2))

    return new_im

Define a function, that will get image features from CLIP.

In [ ]:
class ClipEncoderMulti(nn.Module):
    def __init__(self, num_embeds, num_features=image_features_size):
        super().__init__()        
        self.model = clip_model
        self.num_embeds = num_embeds
        self.num_features = num_features

    def forward(self, x):
        # 4x3x288x288 -> 1x4x640
        out = self.model.encode_image(x.view(-1,3,288,288))
        out = out.view(-1, self.num_embeds, self.num_features).float()
#         print(out.shape)
        return out  # Bx4x640

In [ ]:
# clip_model.vision_model.encoder

In [ ]:
# image_encoder = ClipEncoderMulti(4)
# image = Image.open("dataset/img/42953.png").convert("RGB")

# sliced_images = slice_image(image, 288)
# sliced_images = [np.array(preprocess(im)) for im in sliced_images] 

# image = resize_pad_image(image, image_encoder_size)
# image = np.array(preprocess(image))

# sliced_images = [image] + sliced_images
# sliced_images = torch.from_numpy(np.array(sliced_images)).to(device)

# print(sliced_images.shape)
# print(image_encoder(sliced_images).shape)

In [ ]:
# im = Image.open("dataset/img/01576.png").convert("RGB")
# sliced_images = slice_image(im, 288) 
# for img in sliced_images:
#     plt.figure()
#     plt.imshow(img)
#     print(img.size)

Create JsonlDataset class that will load texts and preprocessed images. <b>collate_fn</b> will group data from dataset in a format needed for our pytorch model.

In [ ]:
class JsonlDataset(Dataset):
    def __init__(self, data_path, tokenizer, transforms, max_seq_length):
        self.data = [json.loads(l) for l in open(data_path)]
        self.data_dir = os.path.dirname(data_path)
        self.tokenizer = tokenizer
        self.max_seq_length = max_seq_length

        self.transforms = transforms

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sentence = torch.LongTensor(self.tokenizer.encode(self.data[index]["text"], add_special_tokens=True))
        start_token, sentence, end_token = sentence[0], sentence[1:-1], sentence[-1]
        sentence = sentence[:self.max_seq_length]

        label = torch.FloatTensor([self.data[index]["label"]])

        image = Image.open(os.path.join(self.data_dir, self.data[index]["img"])).convert("RGB")
        sliced_images = slice_image(image, 288)
        sliced_images = [np.array(self.transforms(im)) for im in sliced_images]
        image = resize_pad_image(image, image_encoder_size)
        image = np.array(self.transforms(image))
        
        sliced_images = [image] + sliced_images         
        sliced_images = torch.from_numpy(np.array(sliced_images)).to(device)

        return {
            "image_start_token": start_token,            
            "image_end_token": end_token,
            "sentence": sentence,
            "image": sliced_images,
            "label": label            
        }

    def get_label_frequencies(self):
        label_freqs = Counter()
        for row in self.data:
            label_freqs.update([row["label"]])
        return label_freqs
    
    def get_labels(self):
        labels = []
        for row in self.data:
            labels.append(row["label"])
        return labels

In [ ]:
def collate_fn(batch):
    lens = [len(row["sentence"]) for row in batch]
    bsz, max_seq_len = len(batch), max(lens)

    mask_tensor = torch.zeros(bsz, max_seq_len, dtype=torch.long)
    text_tensor = torch.zeros(bsz, max_seq_len, dtype=torch.long)

    for i_batch, (input_row, length) in enumerate(zip(batch, lens)):
        text_tensor[i_batch, :length] = input_row["sentence"]
        mask_tensor[i_batch, :length] = 1
    
    img_tensor = torch.stack([row["image"] for row in batch])
    tgt_tensor = torch.stack([row["label"] for row in batch])
    img_start_token = torch.stack([row["image_start_token"] for row in batch])
    img_end_token = torch.stack([row["image_end_token"] for row in batch])

    return {'text_tensor' :text_tensor, 'mask_tensor' : mask_tensor, 'img_tensor': img_tensor, 'img_start_token' : img_start_token, 'img_end_token' : img_end_token, 'tgt_tensor' : tgt_tensor}

Define **load_examples** function that will load data described in json dataset into JsonlDataset object.

In [ ]:
def load_examples(tokenizer, evaluate=False):
    path = os.path.join(data_dir, "dev_seen.jsonl" if evaluate else f"train.jsonl")
    transforms = preprocess
    dataset = JsonlDataset(path, tokenizer, transforms, max_seq_length - num_image_embeds - 2)
    print(dataset)
    return dataset

Create functions to load and save model weights.

In [ ]:
def save_checkpoint(save_path, model, valid_loss):

    if save_path == None:
        return
    
    state_dict = {'model_state_dict': model.state_dict(),
                  'valid_loss': valid_loss}
    
    torch.save(state_dict, save_path)
    print(f'Model saved to ==> {save_path}')
    
def load_checkpoint(load_path, model):
    
    if load_path==None:
        return
    
    state_dict = torch.load(load_path, map_location=device)
    print(f'Model loaded from <== {load_path}')
    
    model.load_state_dict(state_dict['model_state_dict'])
    return state_dict['valid_loss']

Needed functions and classes are created, so we can load our models.

In [ ]:
model_name = 'Hate-speech-CNERG/bert-base-uncased-hatexplain'
transformer_config = AutoConfig.from_pretrained(model_name) 
transformer = AutoModel.from_pretrained(model_name, config=transformer_config)
img_encoder = ClipEncoderMulti(num_image_embeds)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=True)

In [ ]:
config = MMBTConfig(transformer_config, num_labels=num_labels, modal_hidden_size=image_features_size)
model = MMBTForClassification(config, transformer, img_encoder)

In [ ]:
for p in model.mmbt.modal_encoder.encoder.model.parameters():
    if p.requires_grad == True:
        print(p)
        break

In [ ]:
# for p in model.parameters():
#     print(p)
#     break

In [ ]:
model.to(device);

In [ ]:
if torch.cuda.device_count() > 1:
    print('multiple devices')
    model = nn.DataParallel(model).to(device)

Load train and evaluation datasets and create dataloaders for these datasets.

In [ ]:
train_dataset = load_examples(tokenizer, evaluate=False)
eval_dataset = load_examples(tokenizer, evaluate=True)   

train_sampler = RandomSampler(train_dataset)
eval_sampler = SequentialSampler(eval_dataset)

train_dataloader = DataLoader(
        train_dataset,
        sampler=train_sampler,
        batch_size=train_batch_size,
        collate_fn=collate_fn
    )


eval_dataloader = DataLoader(
        eval_dataset, 
        sampler=eval_sampler, 
        batch_size=eval_batch_size, 
        collate_fn=collate_fn
    )

In [ ]:
# for x in train_dataloader:
#     print(x['img_tensor'])
#     break

Define model training parameters, optimizer and loss.

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ["bias", 
            "LayerNorm.weight"
           ]
weight_decay = 0.0005

optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]

t_total = (len(train_dataloader) // gradient_accumulation_steps) * num_train_epochs
warmup_steps = t_total // 10

optimizer = MADGRAD(optimizer_grouped_parameters, lr=2e-4)

scheduler = get_linear_schedule_with_warmup(
        optimizer, warmup_steps, t_total
    )

criterion = nn.BCEWithLogitsLoss()

Define evaluation function that will take evaluation dataloader and calculate prediction AUC, <a href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html">F1 score</a> and accuracy.

In [ ]:
def evaluate(model, tokenizer, criterion, dataloader, tres = 0.5): 
    
    # Eval!
    eval_loss = 0.0
    nb_eval_steps = 0
    preds = None
    proba = None
    out_label_ids = None
    for batch in dataloader:
        model.eval()
        for key,values in batch.items():
            batch[key] = values.to(device)
        with torch.no_grad():
            labels = batch['tgt_tensor']
            inputs = {
            "input_ids": batch['text_tensor'],
            "input_modal": batch['img_tensor'],
            "attention_mask": batch['mask_tensor'],
            "modal_start_tokens": batch['img_start_token'],
            "modal_end_tokens": batch['img_end_token'],
            "return_dict": False
            }
            outputs = model(**inputs)
            logits = outputs[0]  # model outputs are always tuple in transformers (see doc)
            tmp_eval_loss = criterion(logits, labels)
            eval_loss += tmp_eval_loss.mean().item()
        nb_eval_steps += 1
        if preds is None:
            preds = torch.sigmoid(logits).detach().cpu().numpy() > tres
            proba = torch.sigmoid(logits).detach().cpu().numpy()
            out_label_ids = labels.detach().cpu().numpy()
        else:            
            preds = np.append(preds, torch.sigmoid(logits).detach().cpu().numpy() > tres, axis=0)
            proba = np.append(proba, torch.sigmoid(logits).detach().cpu().numpy(), axis=0)
            out_label_ids = np.append(out_label_ids, labels.detach().cpu().numpy(), axis=0)
    
    eval_loss = eval_loss / nb_eval_steps

    result = {
        "loss": eval_loss,
        "accuracy": accuracy_score(out_label_ids, preds),
        "AUC": roc_auc_score(out_label_ids, proba),
        "micro_f1": f1_score(out_label_ids, preds, average="micro"),
        "prediction": preds,
        "labels": out_label_ids,
        "proba": proba
    }
    
    return result

In [ ]:
optimizer_step = 0
global_step = 0
train_step = 0
tr_loss, logging_loss = 0.0, 0.0
best_valid_auc = 0.5
global_steps_list = []
train_loss_list = []
val_loss_list = []
val_acc_list = []
val_auc_list = []
eval_every = len(train_dataloader) // 7
running_loss = 0
file_path="/kaggle/working/"

model.zero_grad()

for i in range(num_train_epochs):
    start = time.time()
    print("Epoch", i+1, f"from {num_train_epochs}")
    whole_y_pred=np.array([])
    whole_y_t=np.array([])
    for step, batch in enumerate(train_dataloader):
        model.train()
        running_loss = 0.0 
        for key,values in batch.items():
            batch[key] = values.to(device)
        labels = batch['tgt_tensor']
#         print(batch['img_tensor'].shape)
        inputs = {
            "input_ids": batch['text_tensor'].to(device),
            "input_modal": batch['img_tensor'].to(device),
            "attention_mask": batch['mask_tensor'].to(device),
            "modal_start_tokens": batch['img_start_token'].to(device),
            "modal_end_tokens": batch['img_end_token'].to(device),
            "return_dict": False
        }
        outputs = model(**inputs)
        logits = outputs[0]  # model outputs are always tuple in transformers (see doc)
        loss = criterion(logits, labels)        
        
        if gradient_accumulation_steps > 1:
            loss = loss / gradient_accumulation_steps
            
        loss.backward()
        
        tr_loss += loss.item()
        running_loss += loss.item()
        global_step += 1
        
        if (step + 1) % gradient_accumulation_steps == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            optimizer.step()
            scheduler.step()  # Update learning rate schedule         
            
            optimizer_step += 1
            optimizer.zero_grad()   
                        
        if (step + 1) % eval_every == 0:
            
            average_train_loss = running_loss / eval_every
            train_loss_list.append(average_train_loss)
            global_steps_list.append(global_step)
            running_loss = 0.0  
            
            val_result = evaluate(model, tokenizer, criterion, eval_dataloader)
            
            val_loss_list.append(val_result['loss'])
            val_acc_list.append(val_result['accuracy'])
            val_auc_list.append(val_result['AUC'])
            
            # checkpoint
            if val_result['AUC'] > best_valid_auc:
                best_valid_auc = val_result['AUC']
                val_loss = val_result['loss']
                val_acc = val_result['accuracy']
                model_path = f'{file_path}/model.pth'
                print(f"AUC improved, so saving this model")  
                save_checkpoint(model_path, model, val_result['loss'])              
            
            print("Train loss:", f"{average_train_loss:.4f}", 
                  "Val loss:", f"{val_result['loss']:.4f}",
                  "Val acc:", f"{val_result['accuracy']:.4f}",
                  "AUC:", f"{val_result['AUC']:.4f}",
                 'time:', f'{time.time() - start}')   
    print('\n')

In [ ]:
import pandas as pd
item = {'train loss' : train_loss_list, 'val loss' : val_loss_list, 'val accuracy' : val_acc_list, 'val_auc' : val_auc_list}
df = pd.DataFrame(item)
df.to_csv('/kaggle/working/stats.csv')

Finally we can train our model. We specify minimun needed AUC value in <b>best_valid_auc</b> variable, so if the model achieves higher AUC on validation data than it was specified, we will save that model.

In [ ]:
# optimizer_step = 0
# global_step = 0
# train_step = 0
# tr_loss, logging_loss = 0.0, 0.0
# best_valid_auc = 0
# global_steps_list = []
# train_loss_list = []
# val_loss_list = []
# val_acc_list = []
# val_auc_list = []
# eval_every = len(train_dataloader)
# running_loss = 0
# file_path="/kaggle/working/"

# model.zero_grad()

# for i in range(num_train_epochs):
#     print("Epoch", i+1, f"from {num_train_epochs}")
#     whole_y_pred=np.array([])
#     whole_y_t=np.array([])
#     preds = []
#     out_label_ids = []
#     for step, batch in enumerate(tqdm(train_dataloader)):
#         model.train()
# #         batch = tuple(values.to(device) for key,values in batch.items())
#         for key,values in batch.items():
#             batch[key] = values.to(device)
#         labels = batch['tgt_tensor']
# #         print(batch['img_tensor'].shape)
#         inputs = {
#             "input_ids": batch['text_tensor'].to(device),
#             "input_modal": batch['img_tensor'].to(device),
#             "attention_mask": batch['mask_tensor'].to(device),
#             "modal_start_tokens": batch['img_start_token'].to(device),
#             "modal_end_tokens": batch['img_end_token'].to(device),
#             "return_dict": False
#         }
# #         print(inputs)
#         outputs = model(**inputs)
#         logits = outputs[0]  # model outputs are always tuple in transformers (see doc)
# #         print(logits.shape)
# #         print(logits)
#         loss = criterion(logits, labels)        
        
# #         if gradient_accumulation_steps > 1:
# #             loss = loss / gradient_accumulation_steps
         
#         optimizer.zero_grad()
#         loss.backward()
        
#         tr_loss += loss.item()
#         running_loss += loss.item()
#         global_step += 1
#         preds.append((torch.sigmoid(logits).detach().cpu().numpy() > 0.5).astype(int))
            
#         out_label_ids.append(labels.detach().cpu().numpy())
# #         print(preds,out_label_ids)
# #         if (step + 1) % gradient_accumulation_steps == 0:
#         torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
#         optimizer.step()
#     scheduler.step()  # Update learning rate schedule         
            
            
            
           
                        
# #         if (step + 1) % eval_every == 0:
            
#     average_train_loss = running_loss / eval_every
#     train_loss_list.append(average_train_loss)
#     train_accuracy = accuracy_score(out_label_ids, preds)
# #         global_steps_list.append(global_step)
#     running_loss = 0.0  
            
#     val_result = evaluate(model, tokenizer, criterion, eval_dataloader)
            
#     val_loss_list.append(val_result['loss'])
#     val_acc_list.append(val_result['accuracy'])
#     val_auc_list.append(val_result['AUC'])
            
#             # checkpoint
#     if val_result['AUC'] > best_valid_auc:
#                 best_valid_auc = val_result['AUC']
#                 val_loss = val_result['loss']
#                 val_acc = val_result['accuracy']
#                 model_path = f'{file_path}/model-embs{num_image_embeds}-seq{max_seq_length}-auc{best_valid_auc:.3f}-loss{val_loss:.3f}-acc{val_acc:.3f}.pth'
#                 print(f"AUC improved, so saving this model")  
#                 save_checkpoint(model_path, model, val_result['loss'])              
            
#     print("Train loss:", f"{average_train_loss:.4f}", 
#           'train accuracy:', f'{train_accuracy}',
#                   "Val loss:", f"{val_result['loss']:.4f}",
#                   "Val acc:", f"{val_result['accuracy']:.4f}",
#                   "AUC:", f"{val_result['AUC']:.4f}")   
#     print('\n')

In [ ]:
# item = {'train_loss' : train_loss_list, 'train_accuracy' : train_accuracy,'val_loss' : val_result['loss'], 'val_accuracy' : val_result['accuracy'], ' AUC' : val_result['AUC']}

After training is complete, we can visualize results:

In [ ]:
# plt.plot(global_steps_list, val_auc_list)
# plt.grid()
# plt.xlabel('Global Steps')
# plt.ylabel('AUC')
# plt.title('MMBT Area Under the Curve')
# plt.show() 

## Make predictions for submission

Now we can make a prediction for test data. Firstly we will create needed classes and functions for data loading and processing by analogy with the training stage.

In [ ]:
# import gc
# import torch

# # Clear Python's internal garbage collector
# gc.collect()

# # If we're using PyTorch with CUDA, clear the GPU memory as well
# if torch.cuda.is_available():
#     torch.cuda.empty_cache()
#     torch.cuda.synchronize()  # wait for cuda to finish (cuda operations are asynchronous)
#     torch.cuda.ipc_collect() 

In [ ]:
# num_labels = 1
# data_dir = './dataset'
# test_batch_size = 16

## Conclusion

def slice_image(im, desired_size):
    '''
    Resize and slice image
    '''
    old_size = im.size
    ratio = float(desired_size)/min(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])
    im = im.resize(new_size, Image.ANTIALIAS)    
    ar = np.array(im)
    images = []
    if ar.shape[0] < ar.shape[1]:
        middle = ar.shape[1] // 2
        half = desired_size // 2
        
        images.append(Image.fromarray(ar[:, :desired_size]))
        images.append(Image.fromarray(ar[:, middle-half:middle+half]))
        images.append(Image.fromarray(ar[:, ar.shape[1]-desired_size:ar.shape[1]]))
    else:
        middle = ar.shape[0] // 2
        half = desired_size // 2
        
        images.append(Image.fromarray(ar[:desired_size, :]))
        images.append(Image.fromarray(ar[middle-half:middle+half, :]))
        images.append(Image.fromarray(ar[ar.shape[0]-desired_size:ar.shape[0], :]))

    return imagesIn this article I tried to describe in detail the concept and implementation of the approach that we used in Hateful Memes Competition from Facebook. The tasks that the competition set for us turned out to be extremely interesting and we had a lot of fun developing our approach to solve these tasks. I hope you enjoyed reading this article too. 

I also want to mention that to obtain maximum AUC, we combined the prediction of several variants of this model trained with different loss parameters and different augmentation options. But that's a topic for an entirely different article. 